In [ ]:
import sys
sys.path.append("..")
from pathlib import Path
import scraper
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib import parse

In [ ]:
data_dir = Path("../data")
format_name = "T20I"
format_dir = data_dir / format_name


In [ ]:
# sanity check
comms = scraper.scrape_game(660011, 660235, debug=True)
comms

In [ ]:
for year_path in format_dir.iterdir():
    with ThreadPoolExecutor() as pool:
        futures=[]
        for series_path in year_path.iterdir():
            matches_dir = series_path / "matches"

            if matches_dir.exists():
                for match_path in matches_dir.iterdir():
                    if (match_path / "meta.json").exists() and not (match_path / "ball_by_ball.json").exists():
                        match_meta = scraper.load_json(match_path / "meta.json")
                        match_id = match_meta["match"]["objectId"]
                        series_id = match_meta["match"]["series"]["objectId"]
                        if match_meta["match"]["coverage"] == "Y" and match_meta["match"]["status"] not in ["ABANDONED", "CANCELLED", "POSTPONED"]:
                            futures.append(pool.submit(scraper.scrape_and_save, scraper.scrape_game, match_path / "ball_by_ball.json", False, series_id, match_id, debug=False))
                            print(match_path)
                        else:
                            print("no comms available for {}".format(match_path))
                            
                            
            scraper.think_about_it_take_a_second(3)

        # wait for everything from year to finish to avoid overloading
        [r.result() for r in as_completed(futures)]
                
                    